In [1]:
import os.path
import bs4
import urllib
import datetime
import json

In [2]:
base = 'https://chicago.councilmatic.org/'
suffix = '/search/?selected_facets=topics_exact%3AZoning+Reclassification'
page_query = '&page={num}'
resp = urllib.request.urlopen(base + suffix + page_query.format(num=1))
soup = bs4.BeautifulSoup(resp.read(), "html5lib")

In [3]:
# resp = urllib.request.urlopen('file:///Users/djb326/Dropbox/data_visualizations/zoning_reclass/page1.html')
# soup = bs4.BeautifulSoup(resp.read(), "html5lib")

In [4]:
def get_ords(link):
    ord_info = {}
    ord_info['ord'] = link.contents[0].split()[1]
    ord_info['href'] = link.get('href')
    ord_info['status'] = str(link.next_element.next_element.next_element.string).strip()
    
    cal = link.parent.next_sibling.next_sibling.next_sibling.next_sibling
    date_string = cal.find('i', class_='fa fa-fw fa-calendar-o').next_element.split('-')[0].strip()
    date = [int(x) for x in date_string.split('/')]
    ord_info['date'] = datetime.date(date[2], date[0], date[1])
    ord_info['year'] = ord_info['date'].year
    return ord_info


def get_locs(inner_soup, ord_info):
    '''get locations mentioned in ordinance'''
    locs = []
    for inner_link in inner_soup.find_all('ul', class_='list-unstyled'):
        if str(inner_link.previous_element.previous_element).strip() == 'Locations mentioned':
            for list_item in inner_link.find_all('li'):
                locs.append(list_item.string.strip())
    ord_info['locations'] = locs
    return ord_info['locations']


def get_sponsors(inner_soup):
    sponsors = []
    wards = []
    inner_link = inner_soup.find('div', class_='table-responsive')
    spo = 1
    war = 1
    for col in inner_link.find_all('td'):
        if col.find('a'):
            sponsors.append(col.a.string.strip())
            spo += 1
        if col.string:
            wards.append(col.string.strip())
            war +=1
    return sponsors,wards

In [5]:
ord_list = []
page_num = 0
while True:
    page_num = page_num+1
    print(page_num)
    try:
        resp = urllib.request.urlopen(base + suffix + page_query.format(num=page_num))
        soup = bs4.BeautifulSoup(resp.read(), "html5lib")
        
        '''loop through each ordinance on main search page'''
        for link in soup.find_all('a', class_='small'):
            ord_info = get_ords(link)
            if ord_info['year'] < 2015:
                break
    
            '''go to ordinance page to get additional info'''
            inner_resp = urllib.request.urlopen(base+ord_info['href'])
            inner_soup = bs4.BeautifulSoup(inner_resp.read(), "html5lib")
    
            ord_info['locations'] = get_locs(inner_soup, ord_info)
            sponsors, spo_wards = get_sponsors(inner_soup)
            ord_info['sponsors'] = sponsors
            ord_info['sponsor_ward'] = spo_wards

            '''append dict to list'''

            ord_list.append(ord_info)
            
    except:
        break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


In [6]:
ord_list

[{'ord': 'O2019-278',
  'href': '/legislation/o2019-278/',
  'status': 'Active',
  'date': datetime.date(2019, 1, 23),
  'year': 2019,
  'locations': ['1456 N Western Ave Chicago'],
  'sponsors': ['Committee on Zoning, Landmarks and Building Standards',
   'Chicago City Council'],
  'sponsor_ward': []},
 {'ord': 'O2019-282',
  'href': '/legislation/o2019-282/',
  'status': 'Active',
  'date': datetime.date(2019, 1, 23),
  'year': 2019,
  'locations': ['4404 S Wood St Chicago'],
  'sponsors': ['Committee on Zoning, Landmarks and Building Standards',
   'Chicago City Council'],
  'sponsor_ward': []},
 {'ord': 'O2018-9268',
  'href': '/legislation/o2018-9268/',
  'status': 'Passed',
  'date': datetime.date(2019, 1, 23),
  'year': 2019,
  'locations': ['831 N Damen Ave Chicago'],
  'sponsors': ['Chicago City Council',
   'Committee on Zoning, Landmarks and Building Standards',
   'Committee on Zoning, Landmarks and Building Standards',
   'Chicago City Council'],
  'sponsor_ward': []},
 {'

In [9]:
with open('ord_dicts.txt', 'w') as fout:
#     json.dump(ord_list, fout)
    fout.write(json.dumps(ord_list, indent=4, sort_keys=True, default=str))
